In [28]:
import numpy as np
import networkx as nx
import json
from collections import Counter
import random
import sim
import operator
from heapq import nlargest

In [73]:
n = 10

# Load JSON file
# Change to appropriate graph
filepath = '/Users/velis.christ/Downloads/'
filename50 = 'RR.10.50.json'
with open(filepath+filename50) as f:
    adj_list50 = json.load(f)

# Create a graph
G50 = nx.Graph(adj_list50)

filename51 = 'RR.10.51.json'
with open(filepath+filename51) as f:
    adj_list51 = json.load(f)

# Create a graph
G51 = nx.Graph(adj_list51)

In [ ]:
def triangle_algo(n,G):
    centralities_dict = nx.triangles(G)
    sorted_centralities = nlargest(n, centralities_dict.items(), key=operator.itemgetter(1))
    node_keys = [i[0] for i in sorted_centralities]
    #  ans = sorted(centralities_dict, reverse=True)
    return node_keys

In [ ]:
def closeness_algo(n,G):
    centralities_dict = nx.closeness_centrality(G)
    sorted_centralities = nlargest(n, centralities_dict.items(), key=operator.itemgetter(1))
    node_keys = [i[0] for i in sorted_centralities]
    #  ans = sorted(centralities_dict, reverse=True)
    return node_keys

In [ ]:
cent_dict = nx.closeness_centrality(G)

In [128]:
filepath = '/Users/velis.christ/Downloads/'
def compare_test(adj_list,filename,our_nodes):
    with open(filepath+filename) as f:
        nodes_TA = json.load(f)
    node_mappings = dict()
    node_mappings["TA"] = nodes_TA['TA_target'][1]
    node_mappings["JYP"] = our_nodes
    results = sim.run(adj_list, node_mappings)
    #print(results)
    return results

In [106]:
def compare_best(adj_list, best_seeds, test_seeds):
    node_mappings = dict()
    node_mappings["Best"] = best_seeds
    node_mappings["Test"] = test_seeds
    results = sim.run(adj_list, node_mappings)
    return results

In [131]:
def best_strategy(G50, G51):
    # functions are eigenvector, closeness, triangle
    # returns tuples (a,b), where a*eigenvector+b*closeness+(1-a-b)*triangle that beat both 50 and 51
    filepath = '/Users/velis.christ/Downloads/'
    filename50 = 'RR.10.50-TA_target.json'
    with open(filepath+filename50) as f:
        nodes_TA = json.load(f)
    node_mappings50 = dict()
    node_mappings50["TA"] = nodes_TA['TA_target'][1]
    filename51 = 'RR.10.51-TA_target.json'
    with open(filepath+filename51) as f:
        nodes_TA = json.load(f)
    node_mappings51 = dict()
    node_mappings51["TA"] = nodes_TA['TA_target'][1]

    eig_cent50 = nx.eigenvector_centrality(G50)
    close_cent50 = nx.closeness_centrality(G50)
    triangles50 = nx.triangles(G50)
    
    eig_cent51 = nx.eigenvector_centrality(G51)
    close_cent51 = nx.closeness_centrality(G51)
    triangles51 = nx.triangles(G51)

    vals = []
    for alpha in np.linspace(0,1,num=30):
        for beta in np.linspace(0,1,num=30):
            our_node_mapping50 = {}
            for node in G50.nodes():
                our_node_mapping50[node] = alpha*eig_cent50[node]+beta*close_cent50[node]+(1-alpha-beta)*triangles50[node]
            seeds50 = nlargest(n, our_node_mapping50.items(), key=operator.itemgetter(1))
            nodes50 = [i[0] for i in seeds50]
            result50 = compare_test(adj_list50,filename50,nodes50)

            our_node_mapping51 = {}
            for node in G51.nodes():
                our_node_mapping51[node] = alpha*eig_cent51[node]+beta*close_cent51[node]+(1-alpha-beta)*triangles51[node]
            seeds51 = nlargest(n, our_node_mapping51.items(), key=operator.itemgetter(1))
            nodes51 = [i[0] for i in seeds51]
            result51 = compare_test(adj_list51,filename51,nodes51)
            ta50, our50 = result50["TA"], result50["JYP"]
            ta51, our51 = result51["TA"], result51["JYP"]
            if our50 > ta50 and our51 > ta51:
                vals.append((alpha,beta))
    return vals

In [126]:
def optimize(best_strategy, adj_list50, adj_list51):
    ''' Optimize coeffs (alpha, beta) against best_strategy on graph G '''
    triangles50 = nx.triangles(G50)
    eig_cent50 = nx.eigenvector_centrality(G50)
    close_cent50 = nx.closeness_centrality(G50)

    triangles51 = nx.triangles(G51)
    eig_cent51 = nx.eigenvector_centrality(G51)
    close_cent51 = nx.closeness_centrality(G51)
    
    max_score = 0
    best_alpha = best_strategy[0]
    best_beta = best_strategy[1]

    og_best_mapping50 = dict()
    for node in G50.nodes():
        og_best_mapping50[node] = best_alpha*eig_cent50[node]+best_beta*close_cent50[node]+(1-best_alpha-best_beta)*triangles50[node]
    
    og_best_mapping51 = dict()
    for node in G51.nodes():
        og_best_mapping51[node] = best_alpha*eig_cent51[node]+best_beta*close_cent51[node]+(1-best_alpha-best_beta)*triangles51[node]

    best_seeds50 = nlargest(n, og_best_mapping50.items(), key=operator.itemgetter(1))
    best_nodes50 = [i[0] for i in best_seeds50]
    best_seeds51 = nlargest(n, og_best_mapping51.items(), key=operator.itemgetter(1))
    best_nodes51 = [i[0] for i in best_seeds51]

    for alpha in np.linspace(0,1,num=10):
        for beta in np.linspace(0,1,num=10):
            our_node_mapping50 = {}
            for node in G50.nodes():
                our_node_mapping50[node] = alpha*eig_cent50[node]+beta*close_cent50[node]+(1-alpha-beta)*triangles50[node]
            seeds50 = nlargest(n, our_node_mapping50.items(), key=operator.itemgetter(1))
            nodes50 = [i[0] for i in seeds50]
            result50 = compare_best(adj_list50,best_nodes50,nodes50)

            our_node_mapping51 = {}
            for node in G51.nodes():
                our_node_mapping51[node] = alpha*eig_cent51[node]+beta*close_cent51[node]+(1-alpha-beta)*triangles51[node]
            seeds51 = nlargest(n, our_node_mapping51.items(), key=operator.itemgetter(1))
            nodes51 = [i[0] for i in seeds51]
            result51 = compare_best(adj_list51,best_nodes51,nodes51)
            our50 = result50["Test"]
            our51 = result51["Test"]

            if our50 + our51 > max_score:
                max_score = our50 + our51
                best_alpha = alpha
                best_beta = beta
    return best_alpha, best_beta

In [132]:
tuples = best_strategy(G50, G51)
print(tuples)

[(0.13793103448275862, 0.8620689655172413), (0.1724137931034483, 0.8275862068965517), (0.20689655172413793, 0.7931034482758621), (0.24137931034482757, 0.7586206896551724)]


In [138]:
triangles50 = nx.triangles(G50)
eig_cent50 = nx.eigenvector_centrality(G50)
close_cent50 = nx.closeness_centrality(G50)

triangles51 = nx.triangles(G51)
eig_cent51 = nx.eigenvector_centrality(G51)
close_cent51 = nx.closeness_centrality(G51)

filename50 = 'RR.10.50-TA_target.json'
filename51 = 'RR.10.51-TA_target.json'

alpha, beta = tuples[1]
our_node_mapping50 = {}
for node in G50.nodes():
    our_node_mapping50[node] = alpha*eig_cent50[node]+beta*close_cent50[node]+(1-alpha-beta)*triangles50[node]
seeds50 = nlargest(n, our_node_mapping50.items(), key=operator.itemgetter(1))
nodes50 = [i[0] for i in seeds50]
result50 = compare_test(adj_list50,filename50,nodes50)

our_node_mapping51 = {}
for node in G51.nodes():
    our_node_mapping51[node] = alpha*eig_cent51[node]+beta*close_cent51[node]+(1-alpha-beta)*triangles51[node]
seeds51 = nlargest(n, our_node_mapping51.items(), key=operator.itemgetter(1))
nodes51 = [i[0] for i in seeds51]
result51 = compare_test(adj_list51,filename51,nodes51)

print(result50)
print(result51)


{'TA': 211, 'JYP': 285}
{'TA': 10, 'JYP': 490}


NameError: name 'node_mappings' is not defined

In [116]:
print(optimize(tuples[0], adj_list50, adj_list51))

compare_test(adj_list50, )

(0.6666666666666666, 0.3333333333333333)
